<a href="https://colab.research.google.com/github/gulabpatel/LLMs/blob/main/02_1_chatbot_using_Llama2_%26_lanceDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install  the  packages





In [ ]:
!pip install --quiet openai langchain
!pip install --quiet -U lancedb
!pip install pypdf
!pip install sentence-transformers
!pip install unstructured
!pip install ctransformers

Mount your drive for uploading data & model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


crete folder inside drive & nevigate to it
i have llm folder inside drive so



In [ ]:
%cd /content/drive/llm

Import the packges

In [3]:
import lancedb
import re
import pickle
import requests
import zipfile
from pathlib import Path
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.vectorstores import LanceDB
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter


Let's load the embedding model
The all-mpnet-base-v2 model provides the best quality, while all-MiniLM-L6-v2 is 5 times faster and still offers good quality. you can try different models
we are using all -MiniLM-L6-V2 models for this demo, & we are using cpu for inference

In [ ]:
#load embedding model

embeddings_mini = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                   model_kwargs={'device': 'cpu'})

select anyone below method

In [5]:
# #Load the  multiple pdfs
# pdf_folder_path = '/content/drive/llm/pdf_paths/'

# from langchain.document_loaders import PyPDFDirectoryLoader
# loader = PyPDFDirectoryLoader(pdf_folder_path)
# docs = loader.load()

In [8]:
# load single pdf
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/drive/MyDrive/Gulab Patel/Papers/Two queue model/A queueing Inventory System with two class of customers.pdf")
pages = loader.load_and_split()


 Now that we have our raw documents loaded, we need to pre-process them to generate embeddings:



 Initialize a RecursiveCharacterTextSplitter to split text documents into smaller chunks.
 This helps in processing large text data efficiently.

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
)
documents = text_splitter.split_documents(pages)
embeddings = embeddings_mini

Lets use lanceDB as vectore store database & crete temp table

In [11]:
db = lancedb.connect('/tmp/lancedb')
table = db.create_table("pandas_docs", data=[
    {"vector": embeddings.embed_query("Hello World"), "text": "Hello World", "id": "1"}
], mode="overwrite")
docsearch = LanceDB.from_documents(documents, embeddings, connection=table)

Now, as we are using the Llama 2 quantized model, you need to download and upload it to your Google Drive. You can choose different versions based on your system specifications. For demonstration purposes, we are using a small model, but feel free to explore other options
download the models from Thebloke repo.


https://huggingface.co/TheBloke/Llama-2-7B-GGML/tree/main

download the .bin file & kept in your current directory

In [ ]:
#Loading the model
def load_llm():
    # Load the locally downloaded model here
    llm = CTransformers(
        model = "llama-2-7b-chat.ggmlv3.q2_K.bin",
        model_type="llama",
        max_new_tokens = 512,
        temperature = 0.6
    )
    return llm

qa = RetrievalQA.from_chain_type(llm=load_llm(), chain_type="stuff", retriever=docsearch.as_retriever())


You can ask the quetions to your data & i will give you response

In [ ]:
query = "Give brief of the paper summary?"
qa.run(query)

In [ ]:
query = "What is LayoutParser ?"
qa.run(query)